## Tabla de contenidos 

1. [Contexto del modelo](#id1)

    1.1. [Librerías utilizadas](#id2)

2. [Preparación del dataset](#id3)
    
3. [Construcción del modelo](#id4)

4. [Predicciones del modelo](#id5)

5. [Evaluación](#id6)

    5.1. [Análisis por etiqueta](#id7)

    5.2. [Análisis general](#id8)

    5.3. [Conclusiones](#id9)

6. [Exportación del modelo](#id10)

<div id='id1' />

# 1. Contexto del modelo

<div id='id2' />

## 1.1. Librerías utilizadas

In [10]:
# Tratamiento de datos

import numpy as np
import pandas as pd
import pickle
import warnings
warnings.filterwarnings('ignore')

# Gráficas

import seaborn as sns
import matplotlib as mpl 
import matplotlib.pyplot as plt

# Preprocesamiento y modelado

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

<div id='id3' />

# 2. Preparación del dataset

In [7]:
comments_df_undersampled = pd.read_csv('../01_data/02_processed/comments_df_undersampled.csv')
comments_df_undersampled

id                                       comment_text  \
0      11ec6078ad3cd031  \n\nsee in my userpage the articles ive create...   
1      11c0246ef55ef048  \n\nmatt fax page being deleted\n\nhi you just...   
2      278ec4e59398a50b  this article contains information soursed from...   
3      c2788f8bdaefeb13  24 promo \n\ni was really confused when i saw ...   
4      fa04aa41f79c7884             i agree hes a biased editor 1141791837   
...                 ...                                                ...   
32445  fef4cf7ba0012866  \n\n our previous conversation \n\nyou fucking...   
32446  ff39a2895fc3b40e                  you are a mischievious pubic hair   
32447  ffa33d3122b599d6  your absurd edits \n\nyour absurd edits on gre...   
32448  ffb47123b2d82762  \n\nhey listen dont you ever delete my edits e...   
32449  ffbdbb0483ed0841  and im going to keep posting the stuff u delet...   

       toxic  severe_toxic  obscene  threat  insult  identity_hate  
0          0             0        0       0       0              0  
1          0             0        0       0       0              0  
2          0             0        0       0       0              0  
3          0             0        0       0       0              0  
4          0             0        0       0       0              0  
...      ...           ...      ...     ...     ...            ...  
32445      1             0        1       0       1              1  
32446      1             0        0       0       1              0  
32447      1             0        1       0       1              0  
32448      1             0        0       0       1              0  
32449      1             0        1       0       1              0  

[32450 rows x 8 columns]

In [8]:
comments_df_undersampled.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [9]:
# Realizamos el train-test split

X_train, X_test, y_train, y_test = train_test_split(comments_df_undersampled['comment_text'], comments_df_undersampled[[ 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate']], test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(25960,)
(6490,)
(25960, 6)
(6490, 6)


<div id='id4' />

# 3. Construcción del modelo

In [5]:
# Instanciamos el TfidfVectorizer y del LogisticRegression multinominal con sus correspondientes parámetros

vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1,2), max_features=500000)
classifier = MultiOutputClassifier(LogisticRegression(max_iter=10000))

In [11]:
# Generamos el pipeline con las dos instancias anteriores y lo entrenamos

pipeline = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', classifier)
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(max_features=500000, ngram_range=(1, 2),
                                 stop_words='english')),
                ('classifier',
                 MultiOutputClassifier(estimator=LogisticRegression(max_iter=10000)))])

<div id='id5' />

# 4. Predicciones del modelo

In [12]:
# Realizamos la predicción

y_pred = pipeline.predict(X_test)

<div id='id6' />

# 5. Evaluación del modelo

In [13]:
# Calculamos la matriz de confusión, precisión, recall y categorical accuracy para cada etiqueta

labels = comments_df_undersampled[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

for i, label in enumerate(labels.columns):
    print(f"Resultados para la etiqueta: {label}")
    print(f"Confusion Matrix:\n{confusion_matrix(y_test.iloc[:, i], y_pred[:, i])}")
    print(f"Precision: {precision_score(y_test.iloc[:, i], y_pred[:, i], zero_division=1)}")
    print(f"Recall: {recall_score(y_test.iloc[:, i], y_pred[:, i], zero_division=1)}")
    print(f"Categorical Accuracy: {accuracy_score(y_test.iloc[:, i], y_pred[:, i])}")
    print("-" * 50)

# Calculamos la precisión categórica general, o lo que es lo mismo, que todas las etiquetas deban coincidir

categorical_accuracy = accuracy_score(y_test, y_pred)
print(f"Categorical Accuracy General: {categorical_accuracy}")

Resultados para la etiqueta: toxic
Confusion Matrix:
[[3139  298]
 [ 529 2524]]
Precision: 0.8944011339475549
Recall: 0.8267278087127415
Categorical Accuracy: 0.8725731895223421
--------------------------------------------------
Resultados para la etiqueta: severe_toxic
Confusion Matrix:
[[6121   71]
 [ 233   65]]
Precision: 0.47794117647058826
Recall: 0.2181208053691275
Categorical Accuracy: 0.9531587057010786
--------------------------------------------------
Resultados para la etiqueta: obscene
Confusion Matrix:
[[4671  115]
 [ 566 1138]]
Precision: 0.9082202713487629
Recall: 0.6678403755868545
Categorical Accuracy: 0.8950693374422188
--------------------------------------------------
Resultados para la etiqueta: threat
Confusion Matrix:
[[6395    6]
 [  76   13]]
Precision: 0.6842105263157895
Recall: 0.14606741573033707
Categorical Accuracy: 0.9873651771956856
--------------------------------------------------
Resultados para la etiqueta: insult
Confusion Matrix:
[[4663  271]
 [ 65

In [14]:
# Calcular y mostrar la precisión y el recall combinados
precision_micro = precision_score(y_test, y_pred, average='micro', zero_division=1)
recall_micro = recall_score(y_test, y_pred, average='micro', zero_division=1)
f1_micro = f1_score(y_test, y_pred, average='micro', zero_division=1)

precision_macro = precision_score(y_test, y_pred, average='macro', zero_division=1)
recall_macro = recall_score(y_test, y_pred, average='macro', zero_division=1)
f1_macro = f1_score(y_test, y_pred, average='macro', zero_division=1)

print(f"Precision (Micro): {precision_micro}")
print(f"Recall (Micro): {recall_micro}")
print(f"F1 Score (Micro): {f1_micro}")

print(f"Precision (Macro): {precision_macro}")
print(f"Recall (Macro): {recall_macro}")
print(f"F1 Score (Macro): {f1_macro}")

# Calcular y mostrar la precisión categórica general (es decir, todas las etiquetas deben coincidir)
categorical_accuracy = accuracy_score(y_test, y_pred)
print(f"Categorical Accuracy General: {categorical_accuracy}")

Precision (Micro): 0.858085808580858
Recall (Micro): 0.672607070997413
F1 Score (Micro): 0.7541089268449888
Precision (Macro): 0.7494312335956059
Recall (Macro): 0.4331113209492645
F1 Score (Macro): 0.5161110248188719
Categorical Accuracy General: 0.6583975346687211


<div id='id7' />

### 5.1. Análisis por etiqueta

1. **Toxic**:
   - **Matriz de Confusión**:
     - Verdaderos negativos (TN): 3088
     - Falsos positivos (FP): 349
     - Falsos negativos (FN): 498
     - Verdaderos positivos (TP): 2555
   - **Precisión**: 0.8798
   - **Recall**: 0.8369
   - **Categorical Accuracy**: 0.8695
   - **Evaluación**: El modelo muestra una buena precisión y recall, indicando un buen equilibrio en la identificación de comentarios tóxicos.

2. **Severe Toxic**:
   - **Matriz de Confusión**:
     - TN: 6124
     - FP: 68
     - FN: 237
     - TP: 61
   - **Precisión**: 0.4729
   - **Recall**: 0.2047
   - **Categorical Accuracy**: 0.9530
   - **Evaluación**: La precisión es baja, y el recall es aún menor, lo que indica que el modelo tiene dificultades para identificar correctamente los comentarios severamente tóxicos.

3. **Obscene**:
   - **Matriz de Confusión**:
     - TN: 4686
     - FP: 100
     - FN: 643
     - TP: 1061
   - **Precisión**: 0.9139
   - **Recall**: 0.6227
   - **Categorical Accuracy**: 0.8855
   - **Evaluación**: La precisión es alta, pero el recall podría mejorarse, sugiriendo que el modelo es más preciso que exhaustivo en la detección de obscenidades.

4. **Threat**:
   - **Matriz de Confusión**:
     - TN: 6401
     - FP: 0
     - FN: 80
     - TP: 9
   - **Precisión**: 1.0
   - **Recall**: 0.1011
   - **Categorical Accuracy**: 0.9877
   - **Evaluación**: La precisión perfecta y el bajo recall indican que el modelo rara vez predice amenazas, pero cuando lo hace, es correcto.

5. **Insult**:
   - **Matriz de Confusión**:
     - TN: 4687
     - FP: 247
     - FN: 653
     - TP: 903
   - **Precisión**: 0.7852
   - **Recall**: 0.5803
   - **Categorical Accuracy**: 0.8613
   - **Evaluación**: La precisión y el recall son moderados, sugiriendo que el modelo tiene un rendimiento aceptable en la identificación de insultos, pero hay margen de mejora.

6. **Identity Hate**:
   - **Matriz de Confusión**:
     - TN: 6221
     - FP: 11
     - FN: 225
     - TP: 33
   - **Precisión**: 0.75
   - **Recall**: 0.1279
   - **Categorical Accuracy**: 0.9636
   - **Evaluación**: La precisión es razonable, pero el recall es bajo, indicando que el modelo no detecta bien los comentarios de odio hacia la identidad.

<div id='id8' />

### 5.2. Análisis General

1. **Categorical Accuracy General**: 0.6553
   - Esto indica que solo el 65.5% de las predicciones coinciden completamente con las etiquetas verdaderas.

2. **Micro Average**:
   - **Precisión**: 0.8564
   - **Recall**: 0.6643
   - **F1 Score**: 0.7482
   - **Evaluación**: La precisión es bastante alta, pero el recall es menor, lo que sugiere que el modelo es más confiable en sus predicciones correctas que en la identificación exhaustiva de todas las etiquetas.

3. **Macro Average**:
   - **Precisión**: 0.8003
   - **Recall**: 0.4123
   - **F1 Score**: 0.4923
   - **Evaluación**: Estas métricas promedian el rendimiento en todas las etiquetas. La diferencia notable entre precisión y recall muestra que el modelo tiene dificultades con algunas etiquetas más que con otras.

<div id='id9' />

### 5.3. Conclusión

El modelo tiene un buen rendimiento en la detección de etiquetas como "toxic" y "obscene" con una alta precisión y un recall razonable. Sin embargo, tiene dificultades para identificar correctamente las etiquetas menos frecuentes como "severe_toxic", "threat" e "identity_hate", lo que es evidente en los bajos valores de recall para estas categorías. Las métricas promedio (micro y macro) confirman que, aunque el modelo es generalmente preciso, necesita mejorar en la detección de etiquetas menos comunes.

<div id='id10' />

# 6. Exportación del modelo

In [25]:
# Definimos el nombre del archivo
filename = '03_segundo_modelo.pkl'

# Guardamos el pipeline en un archivo pickle

with open(filename, 'wb') as file:
    pickle.dump(pipeline, file)